In [1]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import RobustScaler

flights_train_validation_ = pd.read_csv('flights_train.csv')
flights_test_ = pd.read_csv('flights_test.csv')
airports_ = pd.read_csv('airports.csv')

In [2]:
# Display the dataframe
pd.set_option('display.max_columns', 100)  # or 1000
pd.set_option('display.max_rows', 100)  # or 1000
pd.set_option('display.max_colwidth', 100)  # or 199

import warnings
warnings.filterwarnings("ignore")

In [26]:
airports = airports_.copy()
flights_train_validation = flights_train_validation_.copy()
flights_test = flights_test_.copy()

In [27]:
flights_train_validation.head()

,id,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY
0,0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,21.0,15.0,205.0,1448,430,-22.0
1,1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,12.0,14.0,280.0,2330,750,-9.0
2,2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,16.0,34.0,286.0,2296,806,5.0
3,3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,15.0,30.0,285.0,2342,805,-9.0
4,4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,11.0,35.0,235.0,1448,320,-21.0


In [28]:
origin_rank = flights_train_validation.groupby('ORIGIN_AIRPORT').ARRIVAL_DELAY.mean().sort_values(ascending=True).index.tolist()
destination_rank = flights_train_validation.groupby('DESTINATION_AIRPORT').ARRIVAL_DELAY.mean().sort_values(ascending=True).index.tolist()
airline_rank = flights_train_validation.groupby('AIRLINE').ARRIVAL_DELAY.mean().sort_values(ascending=True).index.tolist()
origin_rank = {p:i for i, p in enumerate(origin_rank)}
destination_rank = {p:i for i, p in enumerate(destination_rank)}
airline_rank = {p:i for i, p in enumerate(airline_rank)}

In [29]:
def preprocessing(df):
    df.insert(13, "DEPARTURE_DELAY", (df["DEPARTURE_TIME"] - df["SCHEDULED_DEPARTURE"]))
    df.insert(3, "O_RANK", df["ORIGIN_AIRPORT"].map(origin_rank))
    df.insert(3, "D_RANK", df["DESTINATION_AIRPORT"].map(destination_rank))
    df.insert(3, "A_RANK", df["AIRLINE"].map(airline_rank))

    # Features about delay time
#     df["average_delay_origin_airport"] = flights_train_validation.groupby(["ORIGIN_AIRPORT"])["ARRIVAL_DELAY"].transform("mean").astype(np.float16)
#     df["average_delay_arrival_airport"] = flights_train_validation.groupby(["DESTINATION_AIRPORT"])["ARRIVAL_DELAY"].transform("mean").astype(np.float16)
#     df["average_delay_airline"] = flights_train_validation.groupby(["AIRLINE"])["ARRIVAL_DELAY"].transform("mean").astype(np.float16)
    
#     df["sd_delay_origin_airport"] = flights_train_validation.groupby(["ORIGIN_AIRPORT"])["ARRIVAL_DELAY"].transform(np.std).astype(np.float16)
#     df["sd_delay_arrival_airport"] = flights_train_validation.groupby(["DESTINATION_AIRPORT"])["ARRIVAL_DELAY"].transform(np.std).astype(np.float16)
#     df["sd_delay_airline"] = flights_train_validation.groupby(["AIRLINE"])["ARRIVAL_DELAY"].transform(np.std).astype(np.float16)
    
    # Features about departure delay 
    # df["avg_dep_delay_by_origin"] = flights_train_validation.groupby(["ORIGIN_AIRPORT"])["DEPARTURE_DELAY"].transform("mean").astype(np.float16)
    # df["avg_dep_delay_by_dest"] = flights_train_validation.groupby(["DESTINATION_AIRPORT"])["DEPARTURE_DELAY"].transform("mean").astype(np.float16)
    # df["avg_dep_delay_by_airline"] = flights_train_validation.groupby(["AIRLINE"])["DEPARTURE_DELAY"].transform("mean").astype(np.float16)
    
    # df["sd_dep_delay_by_origin"] = flights_train_validation.groupby(["ORIGIN_AIRPORT"])["DEPARTURE_DELAY"].transform("std").astype(np.float16)
    # df["sd_dep_delay_by_dest"] = flights_train_validation.groupby(["DESTINATION_AIRPORT"])["DEPARTURE_DELAY"].transform("std").astype(np.float16)
    # df["sd_dep_delay_by_airline"] = flights_train_validation.groupby(["AIRLINE"])["DEPARTURE_DELAY"].transform("std").astype(np.float16)
    
    # Binary feature: departure delay yes/no 
    # df["dep_delay_yn"] = df["DEPARTURE_DELAY"].apply(lambda x: 1 if x > 0 else 0)
    
    df = df.sort_values(by="id").set_index("id")
    return df

flights_train_validation = preprocessing(flights_train_validation)
flights_test = preprocessing(flights_test)

In [30]:
# Drop unuseful cols: "YEAR" (year = 2015 for every row), "id", "TAIL_NUMBER" (high cardinality),
# "ORIGIN_AIRPORT", "DESTINATION_AIRPORT" (high cardinality, already have longitude and latitude)
flights_train_validation.drop(["YEAR", "TAIL_NUMBER", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT"], axis=1, inplace=True)
flights_test.drop(["YEAR", "TAIL_NUMBER", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT"], axis=1, inplace=True)

In [31]:
# Encoding airline
# 1. One-hot encoding
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

OH_cols_train_val = pd.DataFrame(OH_encoder.fit_transform(flights_train_validation[["AIRLINE"]]))
OH_cols_train_val.columns=OH_encoder.get_feature_names(["AIRLINE"])

OH_cols_test = pd.DataFrame(OH_encoder.transform(flights_test[["AIRLINE"]]))
OH_cols_test.columns=OH_encoder.get_feature_names(["AIRLINE"])

# One-hot encoding removed index; put it back
OH_cols_train_val.index = flights_train_validation.index
OH_cols_test.index = flights_test.index

# Add one-hot encoded columns to numerical features
flights_train_validation = pd.concat([flights_train_validation, OH_cols_train_val], axis=1)
flights_test = pd.concat([flights_test, OH_cols_test], axis=1)

# Drop AIRLINE col
flights_train_validation.drop("AIRLINE", axis=1, inplace=True)
flights_test.drop("AIRLINE", axis=1, inplace=True)

In [24]:
flights_train_validation.head()

,MONTH,A_RANK,D_RANK,O_RANK,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_TIME,TAXI_OUT,DEPARTURE_DELAY,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,avg_dep_delay_by_origin,avg_dep_delay_by_dest,avg_dep_delay_by_airline,AIRLINE_AA,AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,0,42,33,1,4,98,5,2354.0,21.0,2349.0,15.0,205.0,1448,430,-22.0,-27.671875,1.354492,-5.179688,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,7,178,188,1,4,2336,10,2.0,12.0,-8.0,14.0,280.0,2330,750,-9.0,-2.101562,12.890625,9.242188,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,3,57,191,1,4,840,20,18.0,16.0,-2.0,34.0,286.0,2296,806,5.0,1.613281,4.785156,1.775391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1,7,155,188,1,4,258,20,15.0,15.0,-5.0,30.0,285.0,2342,805,-9.0,-2.101562,2.500000,9.242188,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0,23,67,1,4,135,25,24.0,11.0,-1.0,35.0,235.0,1448,320,-21.0,-2.406250,-10.664062,-5.179688,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
flights_train_validation.columns

Index(['MONTH', 'A_RANK', 'D_RANK', 'O_RANK', 'DAY', 'DAY_OF_WEEK',
       'FLIGHT_NUMBER', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'TAXI_OUT',
       'DEPARTURE_DELAY', 'WHEELS_OFF', 'SCHEDULED_TIME', 'DISTANCE',
       'SCHEDULED_ARRIVAL', 'ARRIVAL_DELAY', 'average_delay_origin_airport',
       'average_delay_arrival_airport', 'average_delay_airline',
       'sd_delay_origin_airport', 'sd_delay_arrival_airport',
       'sd_delay_airline', 'avg_dep_delay_by_origin', 'avg_dep_delay_by_dest',
       'avg_dep_delay_by_airline', 'sd_dep_delay_by_origin',
       'sd_dep_delay_by_dest', 'sd_dep_delay_by_airline', 'AIRLINE_AA',
       'AIRLINE_AS', 'AIRLINE_B6', 'AIRLINE_DL', 'AIRLINE_EV', 'AIRLINE_F9',
       'AIRLINE_HA', 'AIRLINE_MQ', 'AIRLINE_NK', 'AIRLINE_OO', 'AIRLINE_UA',
       'AIRLINE_US', 'AIRLINE_VX', 'AIRLINE_WN'],
      dtype='object')

In [65]:
# flights_train_validation.rename(columns={0: "AIRLINE"}, inplace=True)
# flights_test.rename(columns={0: "AIRLINE"}, inplace=True)

In [32]:
flights_train_validation.to_csv("final2_train_val.csv")
flights_test.to_csv("final2_test.csv")

In [68]:
final_test = pd.read_csv("final2_test.csv")
final_test.columns

Index(['id', 'MONTH', 'A_RANK', 'D_RANK', 'O_RANK', 'DAY', 'DAY_OF_WEEK',
       'FLIGHT_NUMBER', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'TAXI_OUT',
       'DEPARTURE_DELAY', 'WHEELS_OFF', 'SCHEDULED_TIME', 'DISTANCE',
       'SCHEDULED_ARRIVAL', 'average_delay_origin_airport',
       'average_delay_arrival_airport', 'average_delay_airline', 'AIRLINE'],
      dtype='object')